# Inputs:

In [1]:
database_name='sakila'

table_name1='actor'

table_name2='film'

table_name3='rental'

file='sample.xlsx'

## Importing packages:

In [2]:
import datetime
starttime=datetime.datetime.now()

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
%%capture
!pip install openpyxl

In [4]:
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.styles import Color
from openpyxl.styles import numbers
from openpyxl.styles import PatternFill
from openpyxl.styles import Font,Border,Side
from openpyxl.utils import get_column_letter

## Custom Functions:

In [5]:
def format_writing(file,sheet_name,color,col_list,start_color,title):
    
    wb=openpyxl.load_workbook(file)
    ws=wb[sheet_name]
    dim=ws.dimensions
    
    ws.sheet_view.showGridLines= False
    
    ws.sheet_view.zoomScale=75
    
    ws.sheet_properties.tabColor=color
    
    reg=Side(border_style="medium",color='000000')
    box=Border(top=reg,bottom=reg,left=reg,right=reg)
    
    for c in ws[dim][:]:
        for p in c:
            if p.value!=None:
                p.border=box
                
    for c in ws[dim][:]:
        for p in c:
            if p.data_type=='n' and p.value!=0.0:
                if '.' in str(p.value):
                    p.number_format=numbers.BUILTIN_FORMATS[4]
                else:
                    p.number_format=numbers.BUILTIN_FORMATS[3]
    
            if p.data_type=='s' and '%' in p.value:
                try:
                    p.value=float(p.value.replace('%',''))/100
                    p.number_format=numbers.BUILTIN_FORMATS[10]
                except ValueError:
                    p.data_type='s'
                    
            elif p.data_type=='s' and '/' in p.value and p.value.replace('/','').isnumeric():
                    p.number_format=numbers.BUILTIN_FORMATS[14]
                
                    
    for c in ws.columns:
        length=max(len(str(cell.value)) for cell in c)
        col=(get_column_letter(c[0].column))
        if length*1.25 >=50:
            ws.column_dimensions[col].width=50
        elif length*1.25<10:
            ws.column_dimensions[col].width=15
        else:
            ws.column_dimensions[col].width=length*1.25
            

    for i in col_list:
        for c in ws[dim][i[0]:i[1]]:
            for p in c:
                if p.value!=None:
                    p.fill=PatternFill('solid',start_color=start_color)
                    p.font=Font(bold=True)
                    
                    
    ws['A1'].value=title
    ws['A1'].font=Font(bold=True,size=11,color='FF0000')
    
    #if 'Notes' in wb.sheetnames:
    #    del wb['Notes']
                    
    wb.save(file)

## Install MY SQL connector

In [6]:
%%capture
!pip install mysql-connector-python

In [7]:
import mysql.connector

In [8]:
mydb = mysql.connector.connect(host="localhost",user="root",password="1234")

In [9]:
print(mydb)

## Running Queries:

In [10]:
cursor = mydb.cursor()

In [11]:
# Define your query
query = "SELECT * FROM {a}.{b};".format(a=database_name,b=table_name1)

In [12]:
print(query)

SELECT * FROM sakila.actor;


In [13]:
cursor.execute(query)

In [14]:
res=cursor.fetchall()

In [15]:
df=pd.DataFrame(res,columns=[i[0] for i in cursor.description])

In [16]:
df

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33
...,...,...,...,...
195,196,BELA,WALKEN,2006-02-15 04:34:33
196,197,REESE,WEST,2006-02-15 04:34:33
197,198,MARY,KEITEL,2006-02-15 04:34:33
198,199,JULIA,FAWCETT,2006-02-15 04:34:33


In [17]:
df['last_update']=pd.to_datetime(df['last_update']).dt.strftime("%m/%d/%Y")

In [18]:
# Define your query
query2 = "SELECT * FROM {a}.{b};".format(a=database_name,b=table_name2)

In [19]:
print(query2)

SELECT * FROM sakila.film;


In [20]:
cursor.execute(query2)

In [21]:
res=cursor.fetchall()

In [22]:
df2=pd.DataFrame(res,columns=[i[0] for i in cursor.description])

In [23]:
df2.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"{Deleted Scenes, Behind the Scenes}",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"{Deleted Scenes, Trailers}",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,None,7,2.99,50,18.99,NC-17,"{Deleted Scenes, Trailers}",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,None,5,2.99,117,26.99,G,"{Behind the Scenes, Commentaries}",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,None,6,2.99,130,22.99,G,{Deleted Scenes},2006-02-15 05:03:42


In [24]:
df2['last_update']=pd.to_datetime(df2['last_update']).dt.strftime("%m/%d/%Y")

In [25]:
with pd.ExcelWriter(file,engine='openpyxl') as writer:
    df.to_excel(writer,sheet_name='Query1',startrow=2,index=None,na_rep='n.a')
    df2.to_excel(writer,sheet_name='Query1',startrow=2,startcol=6,index=None,na_rep='n.a')

In [26]:
col_list=[[0,1]]
format_writing(file,'Query1','92CDDC',col_list,'92CDDC','Query1')

## Sheet 2:

In [27]:
# Define your query
query3 = "SELECT * FROM {a}.{b};".format(a=database_name,b=table_name3)

In [28]:
print(query3)

SELECT * FROM sakila.rental;


In [29]:
cursor.execute(query3)

In [30]:
res=cursor.fetchall()

In [31]:
df3=pd.DataFrame(res,columns=[i[0] for i in cursor.description])

In [32]:
df3.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [33]:
df3['rental_date']=pd.to_datetime(df3['rental_date']).dt.strftime("%m/%d/%Y")
df3['last_update']=pd.to_datetime(df3['last_update']).dt.strftime("%m/%d/%Y")

In [34]:
with pd.ExcelWriter(file,mode='a',engine='openpyxl') as writer:
    df3.to_excel(writer,sheet_name='Query2',startrow=2,index=None,na_rep='n.a')

In [35]:
col_list=[[0,1]]
format_writing(file,'Query2','D8E4BC',col_list,'D8E4BC','Query2')

## Sheet 3:

In [36]:
# Define your query
query4 = "SELECT film_id,title,rating FROM {a}.{b};".format(a=database_name,b=table_name2)

In [37]:
print(query4)

SELECT film_id,title,rating FROM sakila.film;


In [38]:
cursor.execute(query4)

In [39]:
res=cursor.fetchall()

In [40]:
df4=pd.DataFrame(res,columns=[i[0] for i in cursor.description])

In [41]:
df4

,film_id,title,rating
0,1,ACADEMY DINOSAUR,PG
1,2,ACE GOLDFINGER,G
2,3,ADAPTATION HOLES,NC-17
3,4,AFFAIR PREJUDICE,G
4,5,AFRICAN EGG,G
...,...,...,...
995,996,YOUNG LANGUAGE,G
996,997,YOUTH KICK,NC-17
997,998,ZHIVAGO CORE,NC-17
998,999,ZOOLANDER FICTION,R


In [42]:
with pd.ExcelWriter(file,mode='a',engine='openpyxl') as writer:
    df4.to_excel(writer,sheet_name='Query3',startrow=2,index=None,na_rep='n.a')

In [43]:
col_list=[[0,1]]
format_writing(file,'Query3','FFE699',col_list,'FFE699','Query3')

## Adding pivot tables and charts:

In [44]:
%%capture
!pip install aspose-cells-python==23.1.0

In [45]:
import aspose
import aspose.cells as ac
from aspose.cells import Workbook,Style,FileFormatType
from aspose.cells.charts import ChartType,ChartMarkerType,FormattingType
from aspose.cells.drawing import LineType,WeightType,GradientStyleType,FillFormat,GlowEffect
from aspose.cells.pivot import PivotFieldType,PivotFilterType,PivotTableStyleType,PivotConditionFormatScopeType
import aspose.pydrawing as ad

## Query2:

In [46]:
wb=Workbook(file)

ws=wb.worksheets.get('Query2')

a,b=len(df3)+3,len(df3.columns)

pc=ws.pivot_tables

pci=pc.add('A3:G{a}'.format(a=a),'J3',"Timeseries")
                            
np=pc[pci]
                              
np.row_grand=False
                              
np.add_field_to_area(aspose.cells.pivot.PivotFieldType.ROW,list(df3.columns).index('rental_date'))
np.add_field_to_area(aspose.cells.pivot.PivotFieldType.DATA,list(df3.columns).index('inventory_id'))
np.add_field_to_area(aspose.cells.pivot.PivotFieldType.COLUMN,np.data_field);
                              
p1=np.data_fields[0]
p1.number_format='#,###'
                
p2=np.row_fields[0]
p2.number_format='m/d/yyyy'
                              
np.show_in_tabular_form();
np.is_auto_format=True;
np.auto_format_type=22                              
np.pivot_table_style_type=63
                              
idx=ws.slicers.add(np,'P20',np.base_fields[6])
                              
                              
aspose.cells.charts.BackgroundMode=2
ci=ws.charts.add(ChartType.LINE,2,12,18,28)
ch=ws.charts[ci]
ch.pivot_source="Query2!Timeseries"                             
ch.title.text='Timeseries'
ch.title.font.name='Calibri (Body)'
ch.title.font.color=aspose.cells.drawing.ColorHelper.from_ole_color(8355711)                              
ch.category_axis.tick_labels.font.color=aspose.cells.drawing.ColorHelper.from_ole_color(8355711)                             
ch.category_axis.tick_labels.font.name='Calibri (Body)'                                
ch.category_axis.tick_labels.font.size=9                                
ch.category_axis.tick_labels.rotation_angle=90                               
ch.value_axis.tick_labels.font.color=aspose.cells.drawing.ColorHelper.from_ole_color(8355711)                              
ch.value_axis.tick_labels.font.name='Calibri (Body)'                                
ch.value_axis .tick_labels.font.size=9
ch.legend.font.color==aspose.cells.drawing.ColorHelper.from_ole_color(8355711)                              
ch.legend.font.name='Calibri (Body)'                              
ch.title.set_position_auto()                              
ch.plot_area.is_automatic_size=True                              
                              
                              
wb.save(file)                                                            

In [47]:
ws=Workbook(file)

ws=wb.worksheets.get('Query2')

ch=ws.charts[0]
ch.refresh_pivot_data()


ch.chart_area.area.fill_format.fill_type=2
ch.chart_area.area.fill_format.solid_fill.color=aspose.cells.drawing.ColorHelper.from_ole_color(16777215)
ch.plot_area.area.fill_format.fill_type=2
ch.plot_area.area.fill_format.solid_fill.color=aspose.cells.drawing.ColorHelper.from_ole_color(16777215)
ch.plot_area.border.color=aspose.cells.drawing.ColorHelper.from_ole_color(16777215)

ch.value_axis.major_grid_lines.formatting_type=1
ch.value_axis.major_grid_lines.color=aspose.cells.drawing.ColorHelper.from_ole_color(14277081)
ch.value_axis.major_tick_mark=2
ch.value_axis.axis_line.color=aspose.cells.drawing.ColorHelper.from_ole_color(16777215)
ch.category_axis.major_tick_mark=2   

ch.n_series[0].area.fill_format.fill_type=2
ch.n_series[0].area.fill_format.solid_fill.color=aspose.cells.drawing.ColorHelper.from_ole_color(12874308)
ch.n_series[0].area.foreground_color=aspose.cells.drawing.ColorHelper.from_ole_color(12874308)
ch.n_series[0].border.color=aspose.cells.drawing.ColorHelper.from_ole_color(12874308)
ch.n_series[0].marker.marker_style=ChartMarkerType.AUTOMATIC
ch.n_series[0].marker.marker_size_px=4
ch.n_series[0].marker.area.transparency=0.0
ch.n_series[0].marker.area.fill_format.fill_type=2
ch.n_series[0].marker.area.fill_format.solid_fill.color=aspose.cells.drawing.ColorHelper.from_ole_color(12874308)
ch.n_series[0].marker.border.color=aspose.cells.drawing.ColorHelper.from_ole_color(12874308)


ch.value_axis.major_grid_lines.is_visible=False
ch.category_axis.major_grid_lines.is_visible=False

wb.save(file)

## Query3:

In [48]:
wb=Workbook(file)

ws=wb.worksheets.get('Query3')

a,b=len(df4)+3,len(df4.columns)

pc=ws.pivot_tables

pci=pc.add('A3:C{a}'.format(a=a),'F3',"pic1")
                            
np=pc[pci]
                              
np.row_grand=False
                              
np.add_field_to_area(aspose.cells.pivot.PivotFieldType.ROW,list(df4.columns).index('rating'))
np.add_field_to_area(aspose.cells.pivot.PivotFieldType.DATA,list(df4.columns).index('film_id'))
np.add_field_to_area(aspose.cells.pivot.PivotFieldType.COLUMN,np.data_field);
                              
p1=np.data_fields[0]
p1.number_format='#,###'
                
                              
np.show_in_tabular_form();
np.is_auto_format=True;
np.auto_format_type=23                              
np.pivot_table_style_type=67
                                                            
                              
aspose.cells.charts.BackgroundMode=2
ci=ws.charts.add(ChartType.COLUMN,2,8,18,22)
ch=ws.charts[ci]
ch.pivot_source="Query3!pic1"                             
ch.title.text='Movie Count by Rating'
ch.title.font.name='Calibri (Body)'
ch.title.font.color=ad.Color.white                             
ch.category_axis.tick_labels.font.color=ad.Color.white                                
ch.category_axis.tick_labels.font.name='Calibri (Body)'                                
ch.category_axis.tick_labels.font.size=9                                
ch.category_axis.tick_labels.rotation_angle=90                               
ch.value_axis.tick_labels.font.color=ad.Color.white                                  
ch.value_axis.tick_labels.font.name='Calibri (Body)'                                
ch.value_axis .tick_labels.font.size=9
ch.legend.font.color=ad.Color.white                                  
ch.legend.font.name='Calibri (Body)'                              
ch.title.set_position_auto()                              
ch.plot_area.is_automatic_size=True                              
                              
                              
wb.save(file)                                                            

In [49]:
ws=Workbook(file)

ws=wb.worksheets.get('Query3')

ch=ws.charts[0]
ch.refresh_pivot_data()


ch.chart_area.area.fill_format.fill_type=2
ch.chart_area.area.fill_format.solid_fill.color=aspose.cells.drawing.ColorHelper.from_ole_color(4210752)
ch.plot_area.area.fill_format.fill_type=2
ch.plot_area.area.fill_format.solid_fill.color=aspose.cells.drawing.ColorHelper.from_ole_color(4210752)
  

ch.n_series[0].area.fill_format.fill_type=1
ch.n_series[0].border.color=aspose.cells.drawing.ColorHelper.from_ole_color(15105318)


cc=wb.create_cells_color()
cc.color=aspose.cells.drawing.ColorHelper.from_ole_color(15105318)
ch.n_series[0].shape_properties.glow_effect.color=cc
ch.n_series[0].shape_properties.glow_effect.size=5
ch.n_series[0].shape_properties.glow_effect.transparency=0.75
ch.n_series[0].gap_width=315
ch.n_series[0].overlap=-40

ch.value_axis.major_grid_lines.is_visible=False
ch.category_axis.major_grid_lines.is_visible=False

wb.save(file)

In [50]:
wb=Workbook(file)
wb.worksheets.remove_at('Evaluation Warning')
wb.worksheets.remove_at('Evaluation Warning (1)')
wb.worksheets.remove_at('Evaluation Warning (2)')
wb.worksheets.remove_at('Evaluation Warning (3)')
wb.save(file)

In [51]:
endtime=datetime.datetime.now()
diff=endtime-starttime

In [52]:
print("Total Time elapsed: ", round(diff.seconds/60,2))

Total Time elapsed:  0.72
